1. List of documents' IDs

In [ ]:
!pip install requests
!pip install beautifulsoup4

import requests
from bs4 import BeautifulSoup

def get_id(url):
    file_page = BeautifulSoup(requests.get(url).content, 'html.parser')
    doc_id = ','.join([a.text for a in file_page.select('span + .field li:nth-child(2) a')])
    return doc_id

document_list = []
for page_result in range(1, 4):
    link = f"http://archivo.asambleanacional.gob.ec/index.php/search/advanced?page={page_result}&f=&so0=and&sq0=%222019-2021%22&sf0=title&so1=and&sq1=%22ACTAS+ASAMBLEA+NACIONAL%22&sf1=title&limit=100&sort=alphabetic"
    webpage_code = BeautifulSoup(requests.get(link).content, 'html.parser')
    file_urls = [f"http://archivo.asambleanacional.gob.ec{a['href']}" for a in webpage_code.select('.title a')]
    doc_ids = [get_id(url) for url in file_urls]
    document_list.extend(doc_ids)
    print(f"Page: {page_result}")


2. Download documents


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import os

def download_PDFs(url):
    try:
        page = BeautifulSoup(requests.get(url).content, 'html.parser')
        raw_list = [a['href'] for a in page.select('a') if re.search(r'\.pdf$', a['href'])]

        for file_url in raw_list:
            filename = os.path.basename(file_url)
            response = requests.get(file_url)
            with open(filename, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded: {filename}")

    except Exception as e:
        print(f"! Download from {url} failed: {str(e)}")


def get_links(pageNumber):
    url = f"http://archivo.asambleanacional.gob.ec/index.php/search/advanced?page={pageNumber}&f=&so0=and&sq0=%222019-2021%22&sf0=title&so1=and&sq1=%22ACTAS+ASAMBLEA+NACIONAL%22&sf1=title&limit=100&sort=alphabetic"
    webpage_code = BeautifulSoup(requests.get(url).content, 'html.parser')
    a_elements = webpage_code.select('.title a')
    links = [f"http://archivo.asambleanacional.gob.ec{a['href']}" for a in a_elements]
  
    return links


page_numbers = list(range(1, 4))
links = [get_links(page_number) for page_number in page_numbers]
urls = [url for sublist in links for url in sublist]

for url in urls:
    download_PDFs(url)
